In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import json
from textblob import TextBlob

## Abrimos

In [2]:
rutas_archivos = [
    'Google/reviews-estados/review-California/1.json',
    'Google/reviews-estados/review-California/2.json',
    'Google/reviews-estados/review-California/3.json',
    'Google/reviews-estados/review-California/4.json',
    'Google/reviews-estados/review-California/5.json',
    'Google/reviews-estados/review-California/6.json',
    'Google/reviews-estados/review-California/7.json',
    'Google/reviews-estados/review-California/8.json',
    'Google/reviews-estados/review-California/9.json',
    'Google/reviews-estados/review-California/10.json',
    'Google/reviews-estados/review-California/11.json',
    'Google/reviews-estados/review-California/12.json',
    'Google/reviews-estados/review-California/13.json',
    'Google/reviews-estados/review-California/14.json',
    'Google/reviews-estados/review-California/15.json',
    'Google/reviews-estados/review-California/16.json',
    'Google/reviews-estados/review-California/17.json',
    'Google/reviews-estados/review-California/18.json',
]

In [3]:
todos_los_datos = []
# Iterar a través de cada archivo en la lista de rutas
for ruta_archivo in rutas_archivos:
    # Leer cada línea del archivo JSON
    with open(ruta_archivo, 'r') as f:
        for linea in f:
            # Cargar cada línea como un objeto JSON individual
            datos_json = json.loads(linea)
            # Agregar los datos al lista
            todos_los_datos.append(datos_json)


In [4]:
# Convertir a DataFrame de pandas
df_California = pd.json_normalize(todos_los_datos)

## Se eliminan las columnas innecesarias, nulos y duplicados

In [5]:
# Eliminar las columnas 'time', 'resp' y 'pics'
df_California = df_California.drop(['resp', 'pics', 'resp.time', 'resp.text'], axis=1)

In [6]:
df_California = df_California.dropna(how='all')

In [7]:
df_California = df_California.drop_duplicates()


## Se convierte la columna time en una fecha legible para poder analizar la evolucion de las variables a travez del tiempo y se elimina la columna sin transformar

In [8]:
# Convertir la columna 'time' a formato de fecha y hora
df_California['time'] = pd.to_datetime(df_California['time'], unit='ms')

# Crear una nueva columna con la fecha y hora formateada hasta los minutos
df_California['fecha_y_hora'] = df_California['time'].dt.strftime('%d/%m/%Y %H:%M')

In [9]:
df_California = df_California.drop(['time'], axis=1)

## Se realiza el analisis de sentimiento a las reviews

In [10]:
def analizar_sentimiento(review):
    if review is None:
        return 'No message'
    else:
        sentiment = TextBlob(str(review)).sentiment.polarity
        if sentiment > 0:
            return 'Positive'
        elif sentiment < 0:
            return 'Negative'
        else:
            return 'Neutral'

In [11]:
df_California['analisis_sentimiento'] = df_California['text'].apply(analizar_sentimiento)

In [12]:
df_California.head()

,user_id,name,rating,text,gmap_id,fecha_y_hora,analisis_sentimiento
0,108991152262655788985,Song Ro,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,06/01/2021 05:12,Positive
1,111290322219796215751,Rafa Robles,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49,09/02/2021 05:47,Positive
2,112640357449611959087,David Han,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,08/03/2020 05:04,Neutral
3,117440349723823658676,Anthony Kim,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,07/03/2019 05:56,Positive
4,100580770836123539210,Mario Marzouk,5,Great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49,16/05/2017 05:01,Positive


In [15]:
conteo_subcat = df_California['analisis_sentimiento'].value_counts()
conteo_subcat

analisis_sentimiento
Positive      1178181
No message    1148384
Negative       124477
Neutral        119748
Name: count, dtype: int64

## Se eliminan datos anteriores a 2016 y se reinicia el indice

In [13]:
# Convierte la columna 'fecha_y_hora' a tipo datetime especificando el formato
df_California['fecha_y_hora'] = pd.to_datetime(df_California['fecha_y_hora'], format='%d/%m/%Y %H:%M')

# Filtra las filas donde la fecha y hora sea posterior o igual al año 2016
df_California = df_California[df_California['fecha_y_hora'].dt.year >= 2016]

In [14]:
df_California = df_California.reset_index(drop=True)
